In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Model Notebook

The is the notebook for model in age and gender detection. Here, we will extract the data, train it and fit it into our model, and will evaluate our results and optimize the model for better results.

Import the necessary libraries

In [ ]:
import tensorflow as tf
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam,Adagrad,Adadelta,Adamax,RMSprop

Extract data from the dataset

if data has not been downloaded yet, download it from [Here]https://www.kaggle.com/code/eward96/age-and-gender-prediction-on-utkface

In [ ]:
fldr="/kaggle/input/utkface-new/UTKFace"

In [ ]:
import os 
flies=os.listdir(fldr)

Create list of images from dataset

In [ ]:
ages=[]
genders=[]
images=[]

for fle in flies:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    total=fldr+'/'+fle
    print(total)
    image=cv.imread(total)
    
    image=cv.cvtColor(image,cv.COLOR_BGR2RGB)
    image=cv.resize(image,(48,48))
    images.append(image)

Create List of Age and Gender from dataset

In [ ]:
for fle in flies:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    ages.append(age)
    genders.append(gender)

Viewing a Sample

In [ ]:
plt.imshow(images[45])

In [ ]:
print(ages[45])

In [ ]:
print(genders[45])

Saving the images,ages and genders array in system

In [ ]:
images_f=np.array(images)
ages_f=np.array(ages)
genders_f=np.array(genders)

In [ ]:
np.save('/kaggle/working/image.npy',images_f)
np.save('/kaggle/working/ages.npy',ages_f)
np.save('/kaggle/working/genders.npy',genders_f)

Finding number of Male and Female samples respectively

In [ ]:
values,counts=np.unique(genders_f,return_counts=True)
print(counts)

Plotting the number of male and female Samples

In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
gender=['Male','Female']
values=[12391,11317]
ax.bar(gender,values)
plt.show()

Finding the number of Samples in each age group

In [ ]:
values,counts=np.unique(ages_f,return_counts=True)
print(counts)

Converting age counts to List

In [ ]:
val=values.tolist()
cmt=counts.tolist()

Plotting sample agewise

In [ ]:
plt.plot(counts)
plt.xlabel('ages')
plt.ylabel('Frequency')
plt.show()

Defining labels for output

In [ ]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    labels.append(label)
    i=i+1

Creating another list of images from existing one for better learning

In [ ]:
images_f_2=images_f/255
images_f_2.shape

In [ ]:
labels_f=np.array(labels)

In [ ]:
labels_f.shape

Splitting the dataset into test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(images_f_2,labels_f,test_size=0.25)

In [ ]:
Y_train[0:5]

Splitting "Y_test" and "Y_train" further for better understanding

In [ ]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:5]

In [ ]:
Y_train_2[1][0:5]

Defining the Model

In [ ]:
def Convolution(input_tensor,filters):
    x=Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.001))(input_tensor)
    x=Dropout(0.1)(x)         #determining what parameters are dropped to avoid overfitting
    x=Activation('relu')(x)   # to learn patterns and data
    return x

In [ ]:
def model(input_shape):
    inputs=Input((input_shape))
    conv_1=Convolution(inputs,32)
    maxp_1=MaxPool2D(pool_size=(2,2))(conv_1)
    conv_2=Convolution(maxp_1,64)
    maxp_2=MaxPool2D(pool_size=(2,2))(conv_2)
    conv_3=Convolution(maxp_2,128)
    maxp_3=MaxPool2D(pool_size=(2,2))(conv_3)
    conv_4=Convolution(maxp_3,256)
    maxp_4=MaxPool2D(pool_size=(2,2))(conv_4)
    conv_5=Convolution(maxp_4,512)
    maxp_5=MaxPool2D(pool_size=(2,2))(conv_5)
    flatten=Flatten()(maxp_4)
    dense_1=Dense(64,activation='relu')(flatten)
    dense_2=Dense(64,activation='relu')(flatten)
    drop_1=Dropout(0.2)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    output_1=Dense(1,activation='sigmoid',name='sex_out')(drop_1)
    output_2=Dense(1,activation='swish',name='age_out')(drop_2)
    model=tf.keras.Model(inputs=[inputs],outputs=[output_1,output_2])
    model.compile(loss=["binary_crossentropy","mae"],optimizer="Adam",metrics=["accuracy","accuracy"])
    return model

In [ ]:
Model=model((48,48,3))

In [ ]:
Model.summary()

Initialising the Model

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
fle_s='Age_Sex_Detection.model.keras'
checkpoint=ModelCheckpoint(fle_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=25,monitor='val_loss',restore_best_weights='True')
callback_list=[checkpoint,Early_stop]

Running the Model
#This will take some time

In [ ]:
History=Model.fit(X_train,Y_train_2,batch_size=64,validation_data=(X_test,Y_test_2),epochs=250,callbacks=callback_list)

Model Evaluation

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [ ]:
pred=Model.predict(X_test)

In [ ]:
pred[1]

Plotting Loss

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model loss')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

Plotting Accuracy

In [ ]:
plt.plot(History.history['sex_out_accuracy'])
plt.plot(History.history['val_sex_out_accuracy'])
plt.title('Model Accuracy')
plt.xlabel=('Epoch')
plt.ylabel=('Accuracy')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

Plotting gender accuracy

In [ ]:
fig,ax=plt.subplots()
ax.scatter(Y_test_2[1],pred[1])
ax.plot([Y_test_2[1].min(),Y_test_2[1].max()],[Y_test_2[1].min(),Y_test_2[1].max()],'k--',lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

Making Report of the Model(gender only)

In [ ]:
i=0
Pred_1=[]
while(i<len(pred[0])):
    Pred_1.append(int(np.round(pred[0][i])))
    i=i+1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
report=classification_report(Y_test_2[0],Pred_1)

In [ ]:
print(report)

Making Heatmap of the Model(gender only)

In [ ]:
results=confusion_matrix(Y_test_2[0],Pred_1)

In [ ]:
import seaborn as sns 
sns.heatmap(results,annot=True)

Testing Image Samples

In [ ]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    pred_1=Model.predict(np.array([image_test]))
    sex_f=['Male','Female']
    age=int(np.round(pred_1[1][0]))
    sex=int(np.round(pred_1[0][0]))
    print("Predicted Age is "+str(age))
    print("Predicted Gender is "+sex_f[sex])

#test1

In [ ]:
test_image(4,images_f,images_f_2,Model)

#test2

In [ ]:
test_image(23,images_f,images_f_2,Model)

#test3

In [ ]:
test_image(28,images_f,images_f_2,Model)

#test4

In [ ]:
test_image(123,images_f,images_f_2,Model)

#test5

In [ ]:
test_image(150,images_f,images_f_2,Model)